In [1]:
# FIAP - Curso IA para Devs
# Tech Challenge 03 
# Problema: 
#  -- No Tech Challenge desta fase, você precisa executar o fine-tuning de um 
#  -- foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The
#  -- AmazonTitles-1.3MM". O modelo treinado deverá:
#
# Grupo 44
# Francisco Antonio Guilherme
# fagn2013@gmail.com

# Marcelo Lima Gomes
# marcelolimagomes@gmail.com

# FELIPE MORAES DOS SANTOS
# felipe.moraes.santos2014@gmail.com

In [2]:
# Ambiente configurado para treinamento local em um PC com Placa de Vídeo Nvidia RTX-3060 12GB

# Utilizando miniconda, instalado em um Linux Ubuntu conforme orientações do link: https://docs.anaconda.com/miniconda/
# Utilizando miniconda para criação do ambiente do unsloth conforme orientação no link: https://docs.unsloth.ai/get-started/installation/conda-install

# >> Para configurar o ambiente, remova o comentário ("#") das linhas abaixo e execute os comandos.
# Lembre-se de instalar o miniconda previamente.

#!pip install nbformat
#!conda install -c conda-forge ipywidgets
#!conda create --name unsloth_env python=3.10 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y
#!conda activate unsloth_env
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# Esse notebook tem como objetivo realizar a geração de texto, que neste caso trata-se da, geração de avaliações de produtos com base em dados
# disponibilizados por meio do dataset do exercício.

from transformers import TextStreamer
import pandas as pd
import helper # Biblioteca local para apoio ao desenvolvimento
import torch  # Biblioteca fundamental para deep learning, usada para criar e treinar modelos de redes neurais. 
import datasets  # Biblioteca para carregar e preparar conjuntos de dados de diferentes formatos para treinamento de modelos.
from unsloth import is_bfloat16_supported  # Função para verificar se a GPU suporta o formato de ponto flutuante bfloat16, que pode acelerar o treinamento.
from unsloth import FastLanguageModel  # Classe para criar modelos de linguagem otimizados para maior velocidade e eficiência.

print("Versão PyTorch:", torch.__version__)
print("Versão CuDa:", torch.version.cuda)
print("Suporta Precisão Float 16 bits:", is_bfloat16_supported())

In [4]:
max_seq_length = 2048  # Escolha qualquer um! Nós damos suporte automático ao RoPE Scaling internamente!
dtype = None  # None para detecção automática. Float16 para Tesla T4, V100, Bfloat16 para Ampere+
load_in_4bit = False  # Use quantização de 4 bits para reduzir o uso de memória. Pode ser Falso.

# Nome do modelo treinado préviamente, esse modelo passou por dois processos de treinamento com 40 épocas de treinamento e +3000 steps, e 10h de processamento.
# Somente então o modelo convergiu!
model_name = 'tinyllama_finetuned_2'

In [ ]:
raw_model, tokenizer = helper.get_model_by_name(model_name, max_seq_length, dtype, load_in_4bit)  # Carrega modelo na memória

In [ ]:
# Define e carrega as preferências para treinamento do modelo informado
model = FastLanguageModel.get_peft_model(
    # Contem o modelo carregado préviamente em memória
    raw_model,
    # Classificação da decomposição de baixa classificação para fatoração de matrizes de peso. Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    r=16,  
    # Seleciona os módulos para fazer o ajuste fino. Você pode remover alguns para reduzir o uso de memória e tornar o treinamento 
    # mais rápido, mas não sugerimos isso. Apenas treine em todos os módulos!
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    # Fator de escala para a contribuição das matrizes de baixa classificação.
    lora_alpha=16,
    # Probabilidade de zerar elementos em matrizes de baixa classificação para regularização.
    lora_dropout=0,  # Suporta any, mas = 0 é otimizado
    # Deixe como 0 para um treino mais rápido e com menos over-fit!
    bias="none",    # Suporta any, mas = "none" é otimizado
    # [NEW] "unsloth" usa 30% menos VRAM e se adapta a tamanhos de lote 2x maiores!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    # Ativa o Rank-Stabilized LoRA (RSLora).
    use_rslora=True,  
    # Configuração para LoftQ, um método de quantização para os pesos do backbone e inicialização de camadas LoRA.
    loftq_config=None,  
  )

In [ ]:
# Imprime o dispositivo utilizado para treinamento e a quantidade de memória RAM disponível
start_gpu_memory, max_memory = helper.print_start_memory_usage()

In [8]:
# Prompt no padrão Alpaca é, em essência, uma forma estruturada de instrução que você fornece a um modelo de linguagem 
# para direcionar sua resposta. Essa estrutura é projetada para maximizar a qualidade e a relevância das respostas geradas pelo modelo, 
# tornando as interações mais naturais e informativas.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a book review.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Marcador que indica o fim de uma sequência de tokens

#   Função que realiza o chamamento das rotinas de geração de texto por inferência com base no parâmetro "Titulo do produto" 
def predict_text_streamer(model, tokenizer, title):
  # Ativa a inferência nativa do modelo 2x mais rápido
  FastLanguageModel.for_inference(model) 
  # Converte os parâmetros de entrada (Títulos de Produtos) em prompt e em seguida os converte em tokens a serem enviados ao modelo
  inputs = tokenizer([alpaca_prompt.format(title, '')], return_tensors="pt").to("cuda")

  # Gera o texto em formado te Streaming com base no prompt enviado ao modelo
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128, temperature=0.3)

  return None

In [ ]:
# Teste do modelo depois do treinamento

# O Código abaixo carrega na memória o dataset treinado préviamente, de forma aleatória recupera o Título de um produto e o informa para o modelo
# para que a avaliação deste respectivo produto seja gerada pelo modelo.
# Para fins didáticos, em primeiro momento é impresso o Titulo do produto e sua respectiva avaliação.
# Na sequência é impresso o texto da avaliação do produto gerado pelo modelo e assim nos permite comprará-lo com a avaliação original que consta no dataset.

df = pd.read_csv('../data/trn_sample.csv', sep=';')
for _, row in df.sample(frac=1).head(5).iterrows():
  title = row['title']
  content = row['content']
  print(f"Título do produto: [{title}]")
  print(f"Avaliação original:\n {content}")
  print(f"Avaliação gerada pelo modelo:")
  print('---------------------------------------------------------------')
  predict_text_streamer(model, tokenizer, title)
  print('---------------------------------------------------------------')  

In [10]:
# Depois de 40 épocas de treinamento e +3000 steps, o modelo convergiu!